In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os


### Prepare Dataset

In [21]:
# download coco dataset in colab runtime memory
os.makedirs('data/coco')
os.makedirs('data/coco/annotations')

FileExistsError: ignored

In [ ]:
cd data/coco

In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip
!wget http://images.cocodataset.org/zips/val2017.zip
#!wget http://images.cocodataset.org/zips/test2017.zip

In [ ]:
!unzip -qq "/content/data/coco/train2017.zip"
!unzip -qq "/content/data/coco/val2017.zip"
#!unzip -qq "/content/coco/test2017.zip"


### CLIP

In [3]:
cd /content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption

/content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption


In [8]:
!pip install openai-clip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 6.7 MB/s eta 0:00:00
  Created wheel for openai-clip: filename=openai_clip-1.0.1-py3-none-any.whl size=1368605 sha256=8bfa8851e81f5b1a69891b2b9b05629f095cb13da8c814a566b5fb9fc120a7f3
  Stored in directory: /root/.cache/pip/wheels/08/77/8e/8d2f862df6bf7fb4e2007062d2cbaeae49862ec7b56d041229
Successfully built openai-clip


In [9]:
import clip

In [10]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [22]:
# Pre Trained 된 CLIP 모델을 이용, 이미지를 CLIP Encoder 통과
# clip model ViT-B/32 is used in ClipCap paper

!python parse_coco.py --clip_model_type ViT-B/32 --data_path '/content/data/coco' --out_path '/content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption/data/coco' --start_index 360000

566747 captions loaded from json 
checkpoint loaded
checkpoint shape: torch.Size([360000, 512])
  5% 9999/206747 [03:32<56:59, 57.54it/s]index 369999 saved
 10% 19996/206747 [07:01<1:24:06, 37.00it/s]index 379999 saved
 15% 29997/206747 [10:29<50:54, 57.86it/s]index 389999 saved
 19% 39996/206747 [14:04<1:12:58, 38.09it/s]index 399999 saved
 24% 49998/206747 [17:38<1:12:51, 35.86it/s]index 409999 saved
 29% 59997/206747 [21:18<44:07, 55.42it/s]index 419999 saved
 34% 69997/206747 [24:55<57:41, 39.51it/s]index 429999 saved
 39% 79999/206747 [28:25<56:08, 37.63it/s]index 439999 saved
 44% 89998/206747 [32:07<35:56, 54.13it/s]index 449999 saved
 48% 99999/206747 [35:41<32:07, 55.39it/s]index 459999 saved
 53% 109996/206747 [39:29<51:16, 31.45it/s]index 469999 saved
 58% 119998/206747 [43:15<25:37, 56.43it/s]index 479999 saved
 63% 129997/206747 [46:53<33:14, 38.49it/s]index 489999 saved
 68% 139996/206747 [50:34<19:14, 57.83it/s]index 499999 saved
 73% 149995/206747 [54:10<16:52, 56.03it/

### Train Prefix Network

In [3]:
cd /content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption

/content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption


In [4]:
!python train.py --only_prefix --quantisation --prefix_length 40 --bs 64 --data ./data/coco/oscar_split_ViT-B_32_train.pkl --out_dir ./coco_train/ --mapping_type transformer --model_path '/content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption/coco_train/coco_prefix-001.pt'

vocab.json: 100% 1.04M/1.04M [00:00<00:00, 3.19MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 38.0MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 3.31MB/s]
config.json: 100% 665/665 [00:00<00:00, 3.08MB/s]
Data size is 566747
model.safetensors: 100% 548M/548M [00:01<00:00, 290MB/s]
generation_config.json: 100% 124/124 [00:00<00:00, 610kB/s]
Train only prefix
loading model from /content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption/coco_train/coco_prefix-001.pt
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
>>> Training epoch 0
coco_prefix: 100% 8960/8960 [1:07:30<00:00,  2.21it/s, loss=5.15]
>>> Training epoch 1
coco_prefix:   6% 562/8960 [04:13<1:03:02,  2.22it/s, loss=5.2

In [8]:
!python train.py --only_prefix --bs 64 --data ./data/coco/oscar_split_ViT-B_32_train.pkl --out_dir ./coco_train/ --mapping_type transformer

Data size is 566747
Train only prefix
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
>>> Training epoch 0
coco_prefix: 100% 8960/8960 [41:07<00:00,  3.63it/s, loss=2.68]
>>> Training epoch 1
coco_prefix: 100% 8960/8960 [41:02<00:00,  3.64it/s, loss=2.41]
>>> Training epoch 2
coco_prefix: 100% 8960/8960 [40:55<00:00,  3.65it/s, loss=2.01]
>>> Training epoch 3
coco_prefix: 100% 8960/8960 [40:55<00:00,  3.65it/s, loss=2.39]
>>> Training epoch 4
coco_prefix:  39% 3507/8960 [16:02<24:51,  3.66it/s, loss=2.36]Traceback (most recent call last):
  File "/content/drive/MyDrive/Colab Notebooks/DeepLearning/DL_final_project/DL-final_project/CLIP_prefix_caption/train.py", line 408, in <module>
    main()
  File "/content/drive/MyDrive/Colab